In [2]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict, Counter
from operator import itemgetter
from nltk.tokenize import TweetTokenizer
from pprint import pprint

In [3]:
import vincent

In [4]:
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=True):
    tknzr = TweetTokenizer()
    ttokens = tknzr.tokenize(s)
    
    tokens = tokenize(" ".join(ttokens))
    if lowercase:
        #tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
        tokens = [token.lower() for token in tokens if token.isalpha()]
        tokens = set(tokens)
    return list(tokens)
 
tweet = 'RT  Example @marcobonzanini: just an example! :D http://example.com #NLP\u2025'
print(preprocess(tweet))
# ['RT', '@marcobonzanini', ':', 'just', 'an', 'example', '!', ':D', 'http://exampl

[u'rt', u'example', u'just', u'an']


In [5]:
import pandas
import json

#f = open('/Users/alinamazi/Data/results/04_11/Exp2_3h/ua.json','r')


dates_UNITED = []
#import json as simplejson
import json
#tweets_data_file = open('/Users/alinamazi/Data/results/04_09_14_Merge/Exp1/ua_merge.json','r')
tweets_data_file = open('/Users/alinamazi/Data/results/April/Exp1/ua.json','r')
tweets_data_file_aa = open('/Users/alinamazi/Data/results/April/Exp1/aa.json','r')


tweet_data = json.load(tweets_data_file)
tweet_data_aa = json.load(tweets_data_file_aa)

#com = defaultdict(lambda : defaultdict(int))

tweets = pd.DataFrame()

dates_UNITED = map(lambda tweet: tweet['created_at'], tweet_data['tweets'])
dates_AA = map(lambda tweet: tweet['created_at'], tweet_data_aa['tweets'])

In [6]:
# a list of "1" to count the hashtags
ones = [1]*len(dates_UNITED)
ones_aa = [1]*len(dates_AA)


# the index of the series
idx = pandas.DatetimeIndex(dates_UNITED)
# the actual series (at series of 1s for the moment)
UNITED = pandas.Series(ones, index=idx)

idx_aa = pandas.DatetimeIndex(dates_AA)
AA = pandas.Series(ones_aa, index=idx_aa)

 
# Resampling / bucketing
per_minute = UNITED.resample('60Min').sum().fillna(0)
per_minute_aa = AA.resample('60Min').sum().fillna(0)

In [7]:
time_chart = vincent.Line(UNITED)
time_chart = vincent.Line(per_minute)
time_chart.axis_titles(x='Time', y='Freq')
#time_chart.to_json('time_chart.json')

In [15]:
# all the data together
#match_data = dict(UNITED=per_minute)
match_data = dict(UNITED=per_minute, AA=per_minute_aa)

# we need a DataFrame, to accommodate multiple series
all_matches = pandas.DataFrame(data=match_data,
                               index=per_minute.index)
# Resampling as above
all_matches = all_matches.resample('60Min').sum().fillna(0)
 
# and now the plotting
time_chart = vincent.Line(all_matches[['UNITED','AA']])
time_chart.axis_titles(x='Time', y='Freq')
time_chart.legend(title='Matches')
#time_chart.axes.axhline(100,linestyle='--',color='b')
#time_chart.axes((conv('2013-05-14'), color='r', zorder=0))

#time_chart.to_json('time_chart.json')

SyntaxError: invalid syntax (<ipython-input-15-5c1559c6227b>, line 16)

In [66]:
%matplotlib inline
time_chart.to_json('term_freq.json', html_out=True, html_path='chart8.html')
